In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import tqdm

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
def mape(y_true,y_predict):
    return np.sum((np.abs(y_true-y_predict)/y_true)*100)/len(y_true)

In [3]:
def bin_result(value):
    if value<200:
        return 'low'
    elif value<250:
        return 'medium'
    elif value<300:
        return 'high'
    elif value<350:
        return 'very_high'
    else:
        return 'extreme'


In [4]:
def bin_result_int(value):
    if value <150:
        return 0
    elif value<200:
        return 1
    elif value<250:
        return 2
    elif value<300:
        return 3
    elif value<350:
        return 4
    else:
        return 5

In [5]:
custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d")
feature_df = pd.read_csv('csv_data/feature_first_innings.csv',parse_dates=['match_date'],date_parser=custom_date_parser)

In [6]:
cutoff_start_year = '2014'
cutoff_start_date = datetime.strptime(cutoff_start_year+'-01-01','%Y-%m-%d')
feature_df=feature_df[feature_df['match_date']>cutoff_start_date]
#feature_df=feature_df[feature_df['team']=='India']
#feature_df=feature_df[feature_df['match_date']>cutoff_start_date]

In [7]:
feature_df.columns

Index(['match_id', 'match_date', 'team', 'opponent', 'location', 'team_score',
       'opponent_score', 'opponent_base', 'opponent_trend',
       'opponent_trend_predict', 'opponent_mean', 'location_base',
       'location_trend', 'location_trend_predict', 'location_mean',
       'current_base', 'current_trend', 'current_trend_predict',
       'current_mean', 'batsman_mean', 'batsman_max', 'bowler_mean',
       'bowler_max', 'is_train', 'noise', 'runs_scored'],
      dtype='object')

In [8]:
feature_df.dropna(inplace=True)
#feature_df.isnull().sum()

In [9]:
feature_df['result_class'] = feature_df['runs_scored'].apply(bin_result)

In [10]:
#result_df = pd.get_dummies(feature_df[['result_class']])
#result_df['is_train']=feature_df['is_train']

In [11]:
feature_columns = ['team_score',
       'opponent_score', 'opponent_base', 'opponent_trend',
       'opponent_trend_predict', 'opponent_mean', 'location_base',
       'location_trend', 'location_trend_predict', 'location_mean',
       'current_base', 'current_trend', 'current_trend_predict',
       'current_mean', 'batsman_mean', 'batsman_max', 'bowler_mean',
       'bowler_max']


target_column = ['result_class']

In [12]:
feature_df_train = feature_df[feature_df['is_train']==True]
#result_df_train = result_df[result_df['is_train']==True]
feature_df_test = feature_df[feature_df['is_train']==False]
#result_df_test = result_df[result_df['is_train']==False]

In [13]:
scaler = StandardScaler()
x_train = scaler.fit_transform(feature_df_train[feature_columns])
y_train = feature_df_train[target_column]

x_test = scaler.transform(feature_df_test[feature_columns])
y_test = feature_df_test[target_column]

# x_train = np.array(feature_df_train[feature_columns])
# y_train = np.array(feature_df_train[target_column])

# x_test = np.array(feature_df_test[feature_columns])
# y_test = np.array(feature_df_test[target_column])




In [14]:
dc = DecisionTreeClassifier(max_depth=3)
dc.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=3)

In [15]:
y_train_predict = dc.predict(x_train)
y_test_predict = dc.predict(x_test)

In [16]:
accuracy_score(y_train,y_train_predict)

0.44360902255639095

In [17]:
accuracy_score(y_test,y_test_predict)

0.24770642201834864

In [18]:
dc.get_depth()

3

In [19]:
mape(np.array(y_test),np.array(y_test_predict))

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [20]:
y_test

,result_class
723,extreme
724,very_high
725,extreme
726,high
727,medium
...,...
827,high
828,medium
829,very_high
830,extreme


In [21]:
compare_df = pd.DataFrame()
compare_df['actual']=y_test['runs_scored']
compare_df['predicted']=y_test_predict.reshape(-1)

KeyError: 'runs_scored'

In [22]:
compare_df['percentage_error']=abs(compare_df['actual']-compare_df['predicted'])/compare_df['actual']

KeyError: 'actual'

In [23]:
compare_df

""


In [24]:
result_df =pd.concat([feature_df_test,compare_df],axis=1)

In [25]:
result_df[result_df['percentage_error']>0.15]

KeyError: 'percentage_error'

In [26]:
result_df.shape

(109, 27)

# XGBOOST

In [27]:
import xgboost as xg
from xgboost import XGBClassifier

In [28]:
xg_model = XGBClassifier(max_depth=3,n_estimators=40)


In [29]:
xg_model.fit(x_train,y_train)


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=40, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [30]:
y_train_predict_xg = xg_model.predict(x_train)
y_test_predict_xg = xg_model.predict(x_test)

In [31]:
accuracy_score(y_train,y_train_predict_xg)

0.9824561403508771

In [32]:
accuracy_score(y_test,y_test_predict_xg)

0.28440366972477066

In [33]:
#mape(np.array(y_test).reshape(-1),y_test_predict_xg)

In [34]:
#mape(np.array(y_train).reshape(-1),y_train_predict_xg)

In [35]:
from sklearn.naive_bayes import GaussianNB

In [36]:
nb = GaussianNB()
nb.fit(x_train,y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GaussianNB()

In [37]:
y_train_predict_nb = nb.predict(x_train)
y_test_predict_nb = nb.predict(x_test)

In [38]:
accuracy_score(y_train,y_train_predict_nb)

0.37343358395989973

In [39]:
accuracy_score(y_test,y_test_predict_nb)

0.30275229357798167

# nearest neighbour

In [40]:
from sklearn.neighbors import KNeighborsClassifier

In [41]:
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(x_train,y_train)

<ipython-input-41-83f650ba756e>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  neigh.fit(x_train,y_train)


KNeighborsClassifier()

In [42]:
y_train_predict_neigh = neigh.predict(x_train)
y_test_predict_neigh = neigh.predict(x_test)

In [43]:
accuracy_score(y_train,y_train_predict_neigh)

0.5388471177944862

In [44]:
accuracy_score(y_test,y_test_predict_neigh)

0.22935779816513763

# Custom Naive Bayes

In [45]:
nb_columns = ['team','opponent','location']
nb_target = 'result_class'

In [46]:
feature_df_train.columns

Index(['match_id', 'match_date', 'team', 'opponent', 'location', 'team_score',
       'opponent_score', 'opponent_base', 'opponent_trend',
       'opponent_trend_predict', 'opponent_mean', 'location_base',
       'location_trend', 'location_trend_predict', 'location_mean',
       'current_base', 'current_trend', 'current_trend_predict',
       'current_mean', 'batsman_mean', 'batsman_max', 'bowler_mean',
       'bowler_max', 'is_train', 'noise', 'runs_scored', 'result_class'],
      dtype='object')

In [47]:
def get_class_probability(feature_df_train,nb_target):
    classes = list(feature_df_train[nb_target].unique())
    class_probability_dict = {}
    for selected_class in classes:
        class_probability_dict[selected_class]=feature_df_train[feature_df_train[nb_target]==selected_class].shape[0]/feature_df_train.shape[0]
        
    return class_probability_dict

In [48]:
def get_prediction(test_row,feature_df_train,nb_columns,nb_target,class_probability_dict):
    classes = list(feature_df_train[nb_target].unique())
    base_proba = 0
    selected_map = ''
    map_hypo = {}
    for selected_class in classes:
        pr=class_probability_dict[selected_class]
        #print('\t',selelcted_class,'-',pr,'*')
        for col in nb_columns:
            col_value = test_row[col]
            num=feature_df_train[(feature_df_train[col]==col_value)&(feature_df_train[nb_target]==selected_class)].shape[0]+1
            denom = feature_df_train[(feature_df_train[nb_target]==selected_class)].shape[0]+1
            pr=pr*(num/denom)
            
        if pr>base_proba:
            selected_map = selected_class
            base_proba = pr
            
    return selected_map
    
    

In [49]:
class_probability_dict = get_class_probability(feature_df_train,nb_target)
class_probability_dict

{'extreme': 0.09273182957393483,
 'high': 0.2807017543859649,
 'very_high': 0.21303258145363407,
 'medium': 0.24310776942355888,
 'low': 0.17042606516290726}

In [50]:
test_row = {'team':'India','opponent':'Pakistan','location':'Kolkata',}
get_prediction(test_row,feature_df_train,nb_columns,nb_target,class_probability_dict)

'extreme'

In [51]:
feature_df_test['nb_predict']=feature_df_test.apply(lambda x:get_prediction(x,feature_df_train,nb_columns,nb_target,class_probability_dict),axis=1)

<ipython-input-51-bb68ffd33d6b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df_test['nb_predict']=feature_df_test.apply(lambda x:get_prediction(x,feature_df_train,nb_columns,nb_target,class_probability_dict),axis=1)


In [52]:
np.sum(feature_df_test['nb_predict']==feature_df_test[nb_target])/feature_df_test.shape[0]

0.23853211009174313